In [3]:
from selenium import webdriver
import re
import pandas as pd
import os
import urllib.request
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import matplotlib.pyplot as plt
#import cv2
import numpy as np
import pickle
from tqdm import tqdm
from yahoo_fin import stock_info as si

In [4]:
sp = si.tickers_sp500()

In [155]:
option = Options()
option.headless = False
driver = webdriver.Chrome(options=option)

C:\Users\user\AppData\Local\Temp\ipykernel_18736\2623270998.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  option.headless = False


In [160]:
lt = 'https://www.macrotrends.net/stocks/stock-screener'

In [161]:
driver.get(lt)

In [58]:
lvl=[]
stop = 0
while stop==0:
    i = 0
    stop1 = 0
    while stop1==0:
        webElem = driver.find_elements(By.TAG_NAME, "a")
        lvl0 = [(x.text, x.get_attribute('href')) for x in webElem]
        lvl.append(lvl0[779:799])
        i=i+1
        if lvl0[799+i][0] != 'Terms of Service':
            if lvl0[799+i][0] == '...':
                stop1 = 1
                webElem[799+i].click()
            else:
                webElem[799+i].click()
        else:
            stop=1
            stop1=1

In [59]:
len(lvl)

358

In [63]:
lvl0 = [x for y in lvl for x in y]

In [65]:
len(lvl0)

7160

In [96]:
links_dict = {}
for x in lvl0:
    lk = x[1]
    ne = x[0]
    stk_ne = lk.split('/')[5]
    links_dict[stk_ne] = '/'.join(lk.split('/')[:-1])

In [98]:
len(links_dict)

5979

In [97]:
links_dict['UBS']

'https://www.macrotrends.net/stocks/charts/UBS/ubs-group-ag'

In [93]:
cd E:\Stock Selections

E:\Stock Selections


In [99]:
pickle.dump(links_dict,open('links_dict.obj','wb'))

In [136]:
stks=list(links_dict.keys())

In [170]:
dfs = {}
for z in range(536,len(stks)):
    try:
        x = stks[z]
        lt = links_dict[x]+'/pe-ratio'
        driver.get(lt)
        webElem = driver.find_elements(By.CLASS_NAME, "table")
        dt = webElem[0].text.split('\n')[2:]
        th = ['Date','TTM_Net_EPS','PE']
        td = []
        for i in range(len(dt)):
            dtl = dt[i].split(' ')
            if len(dtl)==4:
                td.append([dtl[0],dtl[2],dtl[3]])
            else:
                if dtl[1][0]=='$':
                    td.append([dtl[0],dtl[1],dtl[2]])
                elif dtl[2][0]=='$':
                    td.append([dtl[0],dtl[2],np.nan])
                else:
                    td.append([dtl[0],np.nan,dtl[2]])
        df = pd.DataFrame(td,columns=th)
        lt = links_dict[x]+'/price-sales'
        driver.get(lt)
        webElem = driver.find_elements(By.CLASS_NAME, "table")
        dt = webElem[0].text.split('\n')[2:]
        th1 = ['Date','TTM_Sales_per_Share','Price_to_Sales']
        td1 = []
        for i in range(len(dt)):
            dtl = dt[i].split(' ')
            if len(dtl)==4:
                td1.append([dtl[0],dtl[2],dtl[3]])
            else:
                if dtl[1][0]=='$':
                    td1.append([dtl[0],dtl[1],dtl[2]])
                elif dtl[2][0]=='$':
                    td1.append([dtl[0],dtl[2],np.nan])
                else:
                    td1.append([dtl[0],np.nan,dtl[2]])
        df1 = pd.DataFrame(td1,columns=th1)
        lt = links_dict[x]+'/price-book'
        driver.get(lt)
        webElem = driver.find_elements(By.CLASS_NAME, "table")
        dt = webElem[0].text.split('\n')[2:]
        th2 = ['Date','Book_Value_per_Share','Price_to_Book']
        td2 = []
        for i in range(len(dt)):
            dtl = dt[i].split(' ')
            if len(dtl)==4:
                td2.append([dtl[0],dtl[2],dtl[3]])
            else:
                if dtl[1][0]=='$':
                    td2.append([dtl[0],dtl[1],dtl[2]])
                elif dtl[2][0]=='$':
                    td2.append([dtl[0],dtl[2],np.nan])
                else:
                    td2.append([dtl[0],np.nan,dtl[2]])
        df2 = pd.DataFrame(td2,columns=th2)
        dfs[x] = pd.merge(pd.merge(df,df1,on='Date'),df2,on='Date')
    except:
        continue

In [175]:
dfs['UBER']

,Date,TTM_Net_EPS,PE,TTM_Sales_per_Share,Price_to_Sales,Book_Value_per_Share,Price_to_Book
0,2023-03-31,$-1.73,0.00,$17.06,1.86,NaN,0.00
1,2022-12-31,$-4.69,0.00,$16.17,1.53,NaN,0.00
2,2022-09-30,$-4.54,0.00,$14.86,1.78,NaN,0.00
3,2022-06-30,$-5.21,0.00,$13.20,1.55,NaN,0.00
4,2022-03-31,$-3.30,0.00,$11.11,3.21,NaN,0.00
5,2021-12-31,$-0.32,0.00,$9.17,4.57,NaN,0.00
6,2021-09-30,$-1.30,0.00,$7.93,5.65,NaN,0.00
7,2021-06-30,$-0.64,0.00,$6.98,7.18,NaN,0.00
8,2021-03-31,$-2.24,0.00,$6.07,8.98,NaN,0.00
9,2020-12-31,$-3.88,0.00,$6.39,7.98,NaN,0.00


In [173]:
pickle.dump(dfs, open('dfs.obj','wb'))